In [ ]:
# Maximum Likelihood on Lie Groups

In [ ]:
import src.params as params
params.group = 'SON'
params.manifold = 'S2'

In [ ]:
from src.group import *
from src.manifold import *
from src.metric import *
import src.multiprocess_utils as mpu

from src.plotting import *
%matplotlib inline
plt.rcParams['figure.figsize'] = 13, 10

In [ ]:
# initialize
init(3)

In [ ]:
# setup for testing different versions of stochastic dynamics
q0 = np.array([1e-6,0,0])
g0 = psif(q0)
v0 = np.array([0,1,-1])
p0 = sharppsif(q0,v0)

In [ ]:
# Brownian motion
from src.Brownian_inv import *

sigma.set_value(.3*np.eye(G_dim.eval())) # set metric
K = 64
obss = np.zeros((K,)+g0.shape)
# srng.seed(422)
for i in range(K):
    (ts,gsv) = Brownian_invf(g0,dWsGf())
    obss[i] = gsv[-1]

# plot samples
newfig()
for i in range(K):
    plotg(obss[i])
plt.show()

# on S2
x0 = np.array([0,0,1])
obss_M = np.zeros((K,M_emb_dim.eval(),))
newfig()
plotM()
for i in range(K):
    obss_M[i] = actf(obss[i],x0)
    plotx(obss_M[i])
plt.show()

In [ ]:
# plot estimated density        
newfig()
# plotM(alpha=.4)
plot_density_estimate(obss_M)
plt.show()

In [ ]:
# fiber Brownian motion
from src.quotient import *
from src.Brownian_inv import *
sde_Brownian_inv_fiber = get_sde_fiber(sde_Brownian_inv,lambda g: act(g,x0))
Brownian_inv_fiber = lambda g,dWt: integrate_sde(sde_Brownian_inv_fiber,
                                                  integrator_stratonovich,
                                                  g,dWt)
Brownian_inv_fiberf = theano.function([g,dWt], Brownian_inv_fiber(g,dWt))

(ts,gsv) = Brownian_inv_fiberf(g0,dWsGf())
newfig()
plotg(gsv)
plt.show()

# on S2
newfig()
plotM()
plotx(actsf(gsv.transpose((1,2,0)),x0).T)
plt.show()

In [ ]:
# find g in fiber above x closests to g0
xx = actf(psif(v0),x0)
print(xx)

(lift,mu_lift) = lift_to_fiber(xx,x0)
print(actf(lift,x0))

(ts,gsv) = exptf(VtoLAf(mu_lift))
newfig()
plotg(gsv)
plotg(lift,color='k')
plt.show()

# on S2
newfig()
plotM()
plotx(actsf(gsv.transpose((1,2,0)),x0).T)
plotx(xx,color='k')
plt.show()

In [ ]:
# Delyon/Hu guided process
from src.guided_process import *

# q0 = np.array([1e-6,0,1])
# g0 = psif(q0)
# v0 = q0+np.array([.5,0,0])

# phi = lambda g: -(g-v)
phi = lambda g,v: T.tensordot(inv(sigma),LAtoV(log(invtrns(inv(g),v))),(1,0))
sde_Brownian_inv_guided = get_sde_guided(sde_Brownian_inv,phi,lambda g: sigma)
Brownian_inv_guided = lambda g,v,dWt: integrate_sde(sde_Brownian_inv_guided,
                                                   integrator_stratonovich,
                                                   g,dWt,T.constant(0.),T.constant(0.),v)
v = T.matrix() # target
Brownian_inv_guidedf = theano.function([g,v,dWt], Brownian_inv_guided(g,v,dWt)[:4])

# derivatives
theta = sigma # parameters
def dlog_likelihood(g,v,dWt):
    s = Brownian_inv_guided(g,v,dWt)
    dlog_likelihood = T.grad(s[2][-1],theta)
    
    return [s[0],s[1],s[2],s[3],dlog_likelihood]
dlog_likelihoodf = theano.function([g,v,dWt],dlog_likelihood(g,v,dWt))

v = psif(v0)
(ts,gsv,log_likelihood,log_varphi) = Brownian_inv_guidedf(g0,v,dWsGf())
print("log likelihood: ", log_likelihood[-1], ", log varphi: ", log_varphi[-1])
newfig()
plotg(gsv)
plotg(v,color='k')
plt.show()

# on S2
newfig()
plotM()
plotx(actsf(gsv.transpose((1,2,0)),x0).T)
plotx(actf(v,x0),color='k')
plt.show()

In [ ]:
from src.mle import *
import src.mle as mle
mle.dlog_likelihoodf = dlog_likelihoodf
mle.theta = theta

options['samples_per_obs'] = 1
options['epochs'] = 50
options['learning_rate'] = 1.e-4
options['varphi_update_rate'] = 1.
options['initial'] = np.diag((.08,.1,.2))#.1*np.eye(G_dim.eval())
options['update_v'] = lambda g: g

In [ ]:
log_phis = np.zeros((K,))
try:
    mpu.openPool()
    sol = mpu.pool.imap(partial(bridge_sampling,g0,options['initial'],dWsGf,options),\
                        mpu.inputArgs(obss,log_phis,np.random.randint(1000,size=K)))
    res = list(sol)
    bridges = mpu.getRes(res,0)
    log_varphis = mpu.getRes(res,1)
    log_likelihoods = mpu.getRes(res,2)
    dlog_likelihoods = mpu.getRes(res,3)
except:
    mpu.closePool()
    raise
else:
    mpu.closePool()

# Plot on S2
newfig()
plotM()
colormap = plt.get_cmap('winter')
colors=[colormap(k) for k in np.linspace(0, 1, K)]
for i in range(bridges.shape[0]):
    plotx(actf(obss[i],x0),color=colors[i])
    for j in range(bridges.shape[1]):
        gsv = bridges[i,j]
        plotx(actsf(gsv.transpose((1,2,0)),x0).T,linewidth=.2,color=colors[i])
plt.show()

In [ ]:
try:
    log_likelihoods = np.zeros(options['epochs'])
    thetas = np.zeros((options['epochs'],)+theta.eval().shape)
    log_varphis = np.zeros(K)
    vs = np.array([lift_to_fiber(obs,x0)[0] for obs in obss]) # obs # either obs on G or point in fiber above obs on M
    theta.set_value(options['initial'])
    mpu.openPool()
    for i in range(options['epochs']):
        sol = mpu.pool.imap(partial(bridge_sampling,g0,sigma.eval(),dWsGf,options),\
                            mpu.inputArgs(vs,log_phis,np.random.randint(1000,size=K)))
        res = list(sol)
        bridges = mpu.getRes(res,0)
        log_varphi = np.mean(mpu.getRes(res,1),axis=(0,1))
        log_likelihood = np.mean(mpu.getRes(res,2),axis=(0,1))
        dlog_likelihood = np.mean(mpu.getRes(res,3),axis=(0,1))
        vs = mpu.getRes(res,4)
        
        log_likelihoods[i] = log_likelihood # total log likelihood        
        thetas[i] = theta.eval()
        print("iteration: ", i, ", log-likelihood: ", log_likelihood, ", theta:\n", theta.eval())

        # step, update parameters and varphis
        theta.set_value(theta.eval()+options['learning_rate']*dlog_likelihood)
        log_varphis = (1.-options['varphi_update_rate'])*log_varphis+options['varphi_update_rate']*log_varphi # update log varphis
except:
    mpu.closePool()
    raise
else:
    mpu.closePool()

## plot
plt.plot(range(options['epochs']),log_likelihoods)
plt.plot(np.tile(range(options['epochs']),(thetas[0].size,1)).T,thetas.reshape(thetas.shape[0],-1))
None